### In this Notebook, I have done the following:
1. Inserting local CSV files (*excel sheets that represent tables*) into databricks DBFS
2. Checking the inferred schema and making sure that everything makes sense and is correct
3. Writing this CSV file into delta format
4. Making sure that I put every CSV file into separate subfolder on the delta (each dataset in separate subfolder). These subfolders act as a storage locations for Delta tables. Each Delta table needs its own directory in DBFS, because delta tables store multiple versions of data ; and each table has its own metadata
5. Creating a **bronze schema** (CREATE SCHEMA command). Schema is a logical container that holds tables and views. 
6. Creating tables with the **USING DELTA** command inside the schema
7. Added a load_date column to table Orders

### Key concept:
I am storing physical data in DBFS (Databricks File System), and also I have a schema (database) which helps in organizing the tables logically. In order to use DBFS (File Storage) I need to use spark commands. But, with Metastore(Schemas & Tables) I organize tables logically for SQL queries

File uploaded to /FileStore/tables/ArticleCategories.csv
File uploaded to /FileStore/tables/Customers.csv
File uploaded to /FileStore/tables/Divisions.csv
File uploaded to /FileStore/tables/OrderDetails.csv
File uploaded to /FileStore/tables/Orders.csv
File uploaded to /FileStore/tables/Products.csv
File uploaded to /FileStore/tables/Shippers.csv
File uploaded to /FileStore/tables/Shipments.csv

In [0]:
# Displaying my CSV files with help of spark
file_path = "/FileStore/tables/ArticleCategories.csv"
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(file_path)

df.dtypes

display(df)


CategoryID,CategoryName,Description
5,Women's Footwear,Ladies Shoes
7,Children's Clothes,3-15 Years
2,Women's Clothes,Fashion for Women
4,Swimwear,Beach clothing
6,Men's Footwear,Men Shoes
1,Men's Clothes,Fashion for Men
8,Baby Clothes,0-3 Years
3,Sportswear,Sports clothing


In [0]:
# Now checking how was schema inferred (what are the datatypes)
df.printSchema()

root
 |-- CategoryID: integer (nullable = true)
 |-- CategoryName: string (nullable = true)
 |-- Description: string (nullable = true)



In [0]:
# 
bronze_layer_path = "dbfs:/FileStore/MedallionArchitecture/bronze/ArticleCategories"
df.write.format("delta").mode("overwrite").save(bronze_layer_path)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze;

In [0]:
%sql
CREATE TABLE bronze.ArticleCategories
USING DELTA
LOCATION "dbfs:/FileStore/MedallionArchitecture/bronze/ArticleCategories"

In [0]:
%sql
SELECT *
FROM bronze.ArticleCategories
ORDER BY CategoryID ASC

CategoryID,CategoryName,Description
1,Men's Clothes,Fashion for Men
2,Women's Clothes,Fashion for Women
3,Sportswear,Sports clothing
4,Swimwear,Beach clothing
5,Women's Footwear,Ladies Shoes
6,Men's Footwear,Men Shoes
7,Children's Clothes,3-15 Years
8,Baby Clothes,0-3 Years


In [0]:
%sql
-- I am adding a column load_date which represent date and time when the tables was added
ALTER TABLE bronze.Orders ADD COLUMN load_date TIMESTAMP;

In [0]:
%sql
-- Setting the value to current_timestamp()
UPDATE bronze.Orders
SET load_date = current_timestamp();

num_affected_rows
6571


In [0]:
# Now I am saving the updated table back to DBFS
df = spark.table("bronze.Orders")

# Write it back to DBFS, ensuring it's stored persistently
df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("dbfs:/FileStore/MedallionArchitecture/bronze/Orders")

In [0]:
%sql
SELECT *
FROM bronze.Customers
LIMIT 20;

CustomerID,CompanyName,ContactName,City,Country,DivisionID,Address,Fax,Phone,PostalCode,StateProvince
62,Roba di Piel,Jorge Alemaio,São Paulo,Brazil,4,"Alameda dos Canàrios, 891",null,(11) 555-1189,05487-020,null
35,El Zapato Rojo,Nicolas Balines,San Cristóbal,Venezuela,4,Carrera 22 con Ave. Carlos Soublette #8-35,(5) 555-1948,(5) 555-1340,5022,null
6,Man Kleider,Herman Hinschler,Mannheim,Germany,1,Forsterstr. 57,0621-08924,0621-08460,68306,null
9,La Legion Mercenaire,Bernard de Gaule,Marseille,France,1,"12, rue des Bouchers",91.24.45.41,91.24.45.40,13008,null
13,Los Sombreros Gigantes,Speedy Gonzales,México D.F.,Mexico,2,El Barrio Chino 12,(5) 555-7293,(5) 555-3392,5021,null
16,The sharped dressed man,Ian Wright,London,UK,1,Berkeley Gardens,null,null,null,null
73,Rode & Vite,Preben Elkjaer,København,Denmark,3,Vinbæltet 34,31133557,31123456,1734,null
52,Kohl Industries AG,Helmuth Klein,Leipzig,Germany,1,Heerstr. 22,null,0342-023176,4179,null
14,Das Alpen Shoe,Alfred Neumann,Bern,Switzerland,1,Hauptstr. 29,null,0452-076545,3012,null
59,Extrawagens,Herbert Bernstorf,Salzburg,Austria,1,Geislweg 14,6562-9723,6562-9722,5020,null
